In [1]:
import numpy as np 
import pandas as pd 

In [2]:
DATASET_PATH = '/kaggle/input/large-purchases-by-the-state-of-ca/PURCHASE ORDER DATA EXTRACT 2012-2015_0.csv'
df = pd.read_csv(DATASET_PATH)

# Explore the dataset
- Creation Date (date): it doesn't have null values, and stored with MM/DD/YYYY format.
- Purchase Date (date): it stored with MM/DD/YYYY, and this date can be back dated or future date in the given dataset, we will remove the future dates cuz it didn't mention in the fields descriptions, and we will ignore the back dates before 1900. 
- Fiscal Year (enum): it doesn't have null values, and stored with YYYY-YYYY format, its value based on creation date, the enum values are `2012-2013`, `2013-2014`, and `2014-2015`.
- LPA Number (str): it stored with SBPxxxx or x-xx-xx-xxC formats, if there is a contract number in this field, the amount is considered contract spend.
- Purchase Order Number (str): it doesn't have null values, and `not unique`.
- Requisition Number (str): its `not unique`
- Acquisition Type (enum): its enum value  `IT Goods`, `NON-IT Goods`, `IT Services`, `NON-IT Services`, `IT Telecommunications`.
- Sub-Acquisition Type (str): there are 25 different types.
- Acquisition Method (str): there are 20 different methods.
- Sub-Acquisition Method (str): there are 16 different sub methods.
- Department Name (str): normalized field.
- Supplier Code (int): normalized field.
- Supplier Name (str)
- Supplier Qualifications (list(str)): list of values.
- Supplier Zip Code (str)
- CalCard (bool): it doesn't have null values, just has two options, NO or YES.
- Item Name (str)
- Item Description (str)
- Quantity (int)
- Unit Price (float)
- Total Price (float): this doesn't include taxes or shipping.
- Classification Codes (list(int)): list of UNSPSC numbers.
- Normalized UNSPSC (int): the first number of Classification Codes.
- Commodity Title (str): based on Normalized UNSPSC number.
- Class (int): based on Normalized UNSPSC number. 
- Class Title (str): based on Normalized UNSPSC number.
- Family (int): based on Normalized UNSPSC number.
- Family Title (str): based on Normalized UNSPSC number.
- Segment (int): based on Normalized UNSPSC number.
- Segment Title (str): based on Normalized UNSPSC number.
- Location (object): it has this format, ZIP\n(Latitude, Longitude)

## When using relational database, the best structure is below:
### Department
- id
- name

### Supplier
- id
- name
- qualifications
- zip_code

### Item
- id
- name
- description
- price

### Acquisition
- id
- type
- sub_type
- method
- sub_method

### Location
- id
- zip_code
- latitude
- longitude

### UNSPSC
- id
- code
- commodity_title
- class
- class_title
- family
- family_title
- segment
- segment_title


### **`Procurement`**
- id
- created_at
- fiscal_year
- purchase_date
- lpa
- order_number
- requisition_number
- cal_card
- department: foreign key
- supplier: foreign key
- acquisition: foreign key
- location: foreign key
- item: foreign key
- quantity
- total_price
- classification_codes
- UNSPSC: foreign key

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346018 entries, 0 to 346017
Data columns (total 31 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Creation Date            346018 non-null  object 
 1   Purchase Date            328582 non-null  object 
 2   Fiscal Year              346018 non-null  object 
 3   LPA Number               92345 non-null   object 
 4   Purchase Order Number    346018 non-null  object 
 5   Requisition Number       14369 non-null   object 
 6   Acquisition Type         346018 non-null  object 
 7   Sub-Acquisition Type     68337 non-null   object 
 8   Acquisition Method       346018 non-null  object 
 9   Sub-Acquisition Method   30896 non-null   object 
 10  Department Name          346018 non-null  object 
 11  Supplier Code            345982 non-null  float64
 12  Supplier Name            345982 non-null  object 
 13  Supplier Qualifications  141745 non-null  object 
 14  Supp

In [4]:
df.head()

,Creation Date,Purchase Date,Fiscal Year,LPA Number,Purchase Order Number,Requisition Number,Acquisition Type,Sub-Acquisition Type,Acquisition Method,Sub-Acquisition Method,...,Classification Codes,Normalized UNSPSC,Commodity Title,Class,Class Title,Family,Family Title,Segment,Segment Title,Location
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,76121504,76121504.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95841\n(38.662263, -121.346136)"
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"91436\n(34.151642, -118.49051)"
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,44103127,44103127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95814\n(38.580427, -121.494396)"


# Rename the columns

In [5]:
# Dictionary mapping current column names to the desired field names
new_column_names = {
    "Creation Date": "creation_date",
    "Purchase Date": "purchase_date",
    "Fiscal Year": "fiscal_year",
    "LPA Number": "lpa_number",
    "Purchase Order Number": "purchase_order_number",
    "Requisition Number": "requisition_number",
    "Acquisition Type": "acquisition_type",
    "Sub-Acquisition Type": "sub_acquisition_type",
    "Acquisition Method": "acquisition_method",
    "Sub-Acquisition Method": "sub_acquisition_method",
    "Department Name": "department_name",
    "Supplier Code": "supplier_code",
    "Supplier Name": "supplier_name",
    "Supplier Qualifications": "supplier_qualifications",
    "Supplier Zip Code": "supplier_zip_code",
    "CalCard": "cal_card",
    "Item Name": "item_name",
    "Item Description": "item_description",
    "Quantity": "quantity",
    "Unit Price": "unit_price",
    "Total Price": "total_price",
    "Classification Codes": "classification_codes",
    "Normalized UNSPSC": "normalized_UNSPSC",
    "Commodity Title": "commodity_title",
    "Class": "class",
    "Class Title": "class_title",
    "Family": "family",
    "Family Title": "family_title",
    "Segment": "segment",
    "Segment Title": "segment_title",
    "Location": "location"
}

# Rename the columns using the dictionary
df = df.rename(columns=new_column_names)

# Display the resulting DataFrame with renamed columns
df.head()

,creation_date,purchase_date,fiscal_year,lpa_number,purchase_order_number,requisition_number,acquisition_type,sub_acquisition_type,acquisition_method,sub_acquisition_method,...,classification_codes,normalized_UNSPSC,commodity_title,class,class_title,family,family_title,segment,segment_title,location
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,76121504,76121504.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95841\n(38.662263, -121.346136)"
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"91436\n(34.151642, -118.49051)"
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,44103127,44103127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95814\n(38.580427, -121.494396)"


# Change the Format of the Location Column

### Original Format (Single Column):
`ZIP\n(Latitude, Longitude)`


### Desired Format (Three Columns):
1. **location_zip**: The ZIP code
2. **location_lat**: The Latitude
3. **location_long**: The Longitude
ing the readability.

In [6]:
# Function to split location into zip, latitude, and longitude
def split_location(location):
    if pd.isnull(location):
        return pd.Series([np.nan, np.nan, np.nan])  # Return np.nan if location is null
    
    try:
        # Split location into zip code and coordinates part
        parts = location.split('\n')
        
        # If there's only one part, treat it as the zip code, and lat/long are np.nan
        if len(parts) == 1:
            return pd.Series([parts[0].strip(), np.nan, np.nan])
        
        # If there are two parts, split the second part into latitude and longitude
        zip_code, coords = parts

        # coords == ''
        if not coords:
            return pd.Series([parts[0].strip(), np.nan, np.nan])
        
        lat, long = coords.strip('()').split(',')
        
        # Return as a series with the parsed values
        return pd.Series([zip_code.strip(), float(lat.strip()), float(long.strip())])
    
    except Exception as e:
        # If any error occurs, return np.nan for all values
        # print(f"Error processing location {location}: {e}")
        return pd.Series([np.nan, np.nan, np.nan])

# Apply the function to the 'location' column
df[['location_zip', 'location_lat', 'location_long']] = df['location'].apply(split_location)


# Drop the 'location' column
df.drop(columns=['location'], inplace=True)

# Display the resulting DataFrame
df.head()

,creation_date,purchase_date,fiscal_year,lpa_number,purchase_order_number,requisition_number,acquisition_type,sub_acquisition_type,acquisition_method,sub_acquisition_method,...,commodity_title,class,class_title,family,family_title,segment,segment_title,location_zip,location_lat,location_long
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95841,38.662263,-121.346136
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91436,34.151642,-118.490510
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95814,38.580427,-121.494396


# Convert the fields that have value format list of words to list of values

In [7]:
# Convert 'supplier_qualifications' to a list of strings, handling null values and 'nan' as a string
df['supplier_qualifications'] = df['supplier_qualifications'].apply(
    lambda x: [qualification for qualification in str(x).split()] if x and str(x).lower() != 'nan' else []  # Remove 'nan' from the list
)

# Convert 'classification_codes' to a list of integers, handling null values and invalid values like 'nan'
df['classification_codes'] = df['classification_codes'].apply(
    lambda x: [int(code) for code in str(x).split()] if x and str(x).lower() != 'nan' else []  # Handle 'nan' and non-numeric values
)

# Display the updated DataFrame
df.head()

,creation_date,purchase_date,fiscal_year,lpa_number,purchase_order_number,requisition_number,acquisition_type,sub_acquisition_type,acquisition_method,sub_acquisition_method,...,commodity_title,class,class_title,family,family_title,segment,segment_title,location_zip,location_lat,location_long
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95841,38.662263,-121.346136
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91436,34.151642,-118.490510
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95814,38.580427,-121.494396


# Map the CalCard values to booleans

In [8]:
# Convert 'cal_card' column to boolean (True for "YES", False for "NO")
df['cal_card'] = df['cal_card'].map({'YES': True, 'NO': False})
df.head()

,creation_date,purchase_date,fiscal_year,lpa_number,purchase_order_number,requisition_number,acquisition_type,sub_acquisition_type,acquisition_method,sub_acquisition_method,...,commodity_title,class,class_title,family,family_title,segment,segment_title,location_zip,location_lat,location_long
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95841,38.662263,-121.346136
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91436,34.151642,-118.490510
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95814,38.580427,-121.494396


# Replace all the 'null' string to null values

In [9]:
# Replace multiple variations of 'nan', 'null', etc., with NaN (null) values
df = df.replace(['nan', 'NaN', 'NAN', 'null', 'None'], np.nan)

# Check the result
df.head()

,creation_date,purchase_date,fiscal_year,lpa_number,purchase_order_number,requisition_number,acquisition_type,sub_acquisition_type,acquisition_method,sub_acquisition_method,...,commodity_title,class,class_title,family,family_title,segment,segment_title,location_zip,location_lat,location_long
0,08/27/2013,NaN,2013-2014,7-12-70-26,REQ0011118,REQ0011118,IT Goods,NaN,WSCA/Coop,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01/29/2014,NaN,2013-2014,NaN,REQ0011932,REQ0011932,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11/01/2013,NaN,2013-2014,NaN,REQ0011476,REQ0011476,IT Services,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95841,38.662263,-121.346136
3,06/13/2014,06/05/2014,2013-2014,NaN,4500236642,NaN,NON-IT Goods,NaN,Informal Competitive,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91436,34.151642,-118.490510
4,03/12/2014,03/12/2014,2013-2014,1-10-75-60A,4500221028,NaN,NON-IT Goods,NaN,Statewide Contract,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95814,38.580427,-121.494396


# Remove the $ from the prices

In [10]:
# Function to remove "$" and convert to number
def clean_currency(value):
    if pd.isnull(value):  # If the value is null, leave it as is
        return value
    return float(value.lstrip('$'))  # Remove '$' and convert to float

# Apply this to the relevant column(s)
df['unit_price'] = df['unit_price'].apply(clean_currency)
df['total_price'] = df['total_price'].apply(clean_currency)

# Convert the fields to the correct types

In [11]:
# Mapping columns to their desired data types
# float not int for number cuz some of the values null
column_types = {
    "fiscal_year": "category",  # Assuming it's categorical
    "acquisition_type": "category", # Assuming it's categorical 
    "supplier_code": "float64",  # Supplier code is likely numerical
    "supplier_qualifications": "object",  # List of strings
    "cal_card": "bool", # Boolean 
    "quantity": "float64",  # Quantity is numeric
    "unit_price": "float64",  # Price should be numeric
    "total_price": "float64",  # Price should be numeric
    "classification_codes": "object",  # List of integers
    "normalized_UNSPSC": "float64", 
    "class": "float64", 
    "family": "float64", 
    "segment": "float64", 
    "location_lat": "float64",  # Latitude is numeric
    "location_long": "float64",  # Longitude is numeric
}

# Apply conversion to each column
for column, dtype in column_types.items():
    if column in df.columns:
        df[column] = df[column].astype(dtype, errors='ignore')

# Convert 'creation_date' and 'purchase_date' to datetime format
df['creation_date'] = pd.to_datetime(df['creation_date'], format='%m/%d/%Y', errors='coerce')
df['purchase_date'] = pd.to_datetime(df['purchase_date'], format='%m/%d/%Y', errors='coerce')



# Display the dataframe to verify the changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346018 entries, 0 to 346017
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   creation_date            346018 non-null  datetime64[ns]
 1   purchase_date            328556 non-null  datetime64[ns]
 2   fiscal_year              346018 non-null  category      
 3   lpa_number               92345 non-null   object        
 4   purchase_order_number    346018 non-null  object        
 5   requisition_number       14369 non-null   object        
 6   acquisition_type         346018 non-null  category      
 7   sub_acquisition_type     68337 non-null   object        
 8   acquisition_method       346018 non-null  object        
 9   sub_acquisition_method   30896 non-null   object        
 10  department_name          346018 non-null  object        
 11  supplier_code            345982 non-null  float64       
 12  supplier_name   

# Remove purchases that happend before 2003 and after 2015

In [12]:
# Filter rows where the purchase_date year is between 2003 and 2015, keeping nulls
df = df[(df['purchase_date'].isna()) | (df['purchase_date'].dt.year.between(2003, 2015))]

# Download the dataset into 2 formats
- CSV: to use it in the Power BI
- JSON: to load it in the MongoDB

In [13]:
df.to_json('purchases.json', orient='records', lines=True)
df.to_csv('purchases.csv', index=False) 